<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebooks/Spark_lab_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

In [2]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2026-01-10 01:48:59--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  21.0MB/s    in 24s     

2026-01-10 01:49:24 (21.5 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



In [3]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark401-Training") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


# LAB SPARK

---

##Mục tiêu cuối:

Bạn có thể tự tin viết – đọc – debug – tối ưu Spark job từ local → cloud (Dataproc / EMR / Synapse)

---



# 🟢 LAB 1 – DATAFRAME CƠ BẢN (FOUNDATION)

## 🎯 Mục tiêu
*	Hiểu DataFrame là gì
*	Hiểu Spark khác Pandas ở điểm nào

---

## 1️⃣ Tạo data mẫu

### 🔹 Spark


In [5]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName("lab1").getOrCreate()

df = spark.createDataFrame(
    [(1, "Alice", 1000),
     (2, "Bob", 2000),
     (3, "Charlie", 3000)],
    ["id", "name", "balance"]
)

## 2️⃣ Xử lý

In [6]:
df2 = df.filter(F.col("balance") > 1500)

## 3️⃣ Giải thích:

*	filter() KHÔNG chạy
*	Spark chỉ ghi lại logical plan
*	Chưa có action → chưa có CPU / RAM dùng

---

## 4️⃣ Action (kích hoạt Spark)

In [7]:
df2.show()

+---+-------+-------+
| id|   name|balance|
+---+-------+-------+
|  2|    Bob|   2000|
|  3|Charlie|   3000|
+---+-------+-------+



### 👉 Lúc này Spark mới:

*	Build DAG
*	Tạo stage
*	Tạo task
*	Thực thi

## 5️⃣ So với Pandas

In [8]:
import pandas as pd

pdf = pd.DataFrame({
    "id": [1,2,3],
    "name": ["Alice","Bob","Charlie"],
    "balance": [1000,2000,3000]
})

pdf[pdf["balance"] > 1500]

,id,name,balance
1,2,Bob,2000
2,3,Charlie,3000


|**Pandas**|**Spark**|
|----------|---------|
|Chạy ngay|Lazy|
|RAM máy|RAM cluster|
|Nhỏ|Rất lớn|

👉 Pandas = tiện, Spark = sống sót

---

## 6️⃣ Công cụ thực tế
* Spark UI
*	PySpark
*	Colab / Local / Databricks


# 🟢 LAB 2 – READ FILE & PARTITION (CỰC QUAN TRỌNG)

## 🎯 Mục tiêu
*	Hiểu partition
*	Biết vì sao file to = chết

---

## 1️⃣ Tạo data (CSV giả lập)


In [9]:
data = [(i, f"user_{i}", i*10) for i in range(1, 100_001)]
df = spark.createDataFrame(data, ["id", "name", "score"])

## 2️⃣ Kiểm tra partition

In [10]:
df.rdd.getNumPartitions()

2

## 3️⃣ Repartition

In [11]:
df2 = df.repartition(8)
df2.rdd.getNumPartitions()

8

## 4️⃣ Giải thích
*	Partition = số task
*	Task = 1 core
*	Ít partition → CPU rảnh
*	Nhiều quá → overhead

---

## 5️⃣ Pandas có không?

* ❌ Không có khái niệm partition
* 👉 Pandas không scale

---

## 6️⃣ Thực tế
*	Dataproc / EMR: partition quyết định cost & SLA
*	Bank review partition trước khi review code


# 🟡 LAB 3 – GROUP BY & SHUFFLE (BƯỚC NGOẶT)

## 🎯 Mục tiêu
*	Thấy shuffle thật sự
*	Biết vì sao job stuck 99%

---

## 1️⃣ Data

In [12]:
df = spark.range(0, 1_000_000).select(
    (F.col("id") % 10).alias("group_id"),
    F.rand().alias("amount")
)

## 2️⃣ GroupBy

In [13]:
agg = df.groupBy("group_id").sum("amount")
agg.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[group_id#17L], functions=[sum(amount#18)])
   +- Exchange hashpartitioning(group_id#17L, 4), ENSURE_REQUIREMENTS, [plan_id=53]
      +- HashAggregate(keys=[group_id#17L], functions=[partial_sum(amount#18)])
         +- Project [(id#16L % 10) AS group_id#17L, rand(6847661680794689400) AS amount#18]
            +- Range (0, 1000000, step=1, splits=2)




## 3️⃣ Giải thích
*	Exchange xuất hiện
*	Shuffle bắt buộc
*	Dữ liệu di chuyển giữa executor

> 👉 Tốn nhất trong Spark

---

## 4️⃣ Tune nhẹ:

In [15]:
spark.conf.set("spark.sql.shuffle.partitions", 10)

agg = df.groupBy("group_id").sum("amount")
agg.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[group_id#17L], functions=[sum(amount#18)])
   +- Exchange hashpartitioning(group_id#17L, 10), ENSURE_REQUIREMENTS, [plan_id=70]
      +- HashAggregate(keys=[group_id#17L], functions=[partial_sum(amount#18)])
         +- Project [(id#16L % 10) AS group_id#17L, rand(6847661680794689400) AS amount#18]
            +- Range (0, 1000000, step=1, splits=2)




Hai kế hoạch thực thi (Physical Plan) mà bạn cung cấp gần như hoàn toàn giống nhau, ngoại trừ một điểm khác biệt duy nhất nằm ở bước Shuffle (Exchange).

Dưới đây là bảng so sánh chi tiết:

1. **Điểm giống nhau**

Cả hai kế hoạch đều thực hiện cùng một logic xử lý:

**Range:** Tạo ra 1.000.000 bản ghi, chia làm 2 partitions ban đầu.

**Project:** Tính toán group_id (bằng cách lấy id % 10) và tạo cột amount ngẫu nhiên.

**Partial Aggregation (HashAggregate):** Spark thực hiện cộng tổng tạm thời ngay tại các partition local để giảm lượng dữ liệu cần truyền qua mạng (shuffle).

**AQE (Adaptive Spark Plan):** Cả hai đều đang bật tính năng tự điều chỉnh kế hoạch, nhưng hiện tại là `isFinalPlan=false` (chưa phải bản chốt cuối cùng khi chạy).

---

2. **Điểm khác biệt duy nhất:**

|**Đặc điểm**|**Kế hoạch 1**|**Kế hoạch 2**|
|------------|--------------|--------------|
|Số lượng Partition khi Shuffle|4 partitions|10 partitions|
|Chi tiết tại Exchange|"hashpartitioning(group_id#17L, 4)"|"hashpartitioning(group_id#17L, 10)"|

---

3. **Phân tích tác động**

Sự khác biệt về con số **4** và **10** này có ý nghĩa quan trọng về hiệu suất dựa trên dữ liệu thực tế của bạn:

**Độ song song (Parallelism):**

Ở kế hoạch 1, giai đoạn tính tổng cuối cùng (Final Aggregate) sẽ chỉ có 4 tác vụ (tasks) chạy song song.

Ở kế hoạch 2, bạn có 10 tác vụ chạy song song. Nếu cluster của bạn có nhiều core, kế hoạch 2 có thể nhanh hơn.

**Phân phối dữ liệu (Data Distribution):**

Trong code của bạn, group_id được tính bằng `id % 10`, nghĩa là có tối đa 10 giá trị group_id duy nhất (từ 0 đến 9).

**Kế hoạch 2 (10 partitions):** Là con số "lý tưởng". Mỗi partition sẽ xử lý chính xác 1 giá trị group_id. Dữ liệu được chia đều tăm tắp.

**Kế hoạch 1 (4 partitions):** Spark phải dùng hàm Hash để gom 10 loại group_id này vào 4 "giỏ". Kết quả là sẽ có giỏ chứa 2 nhóm, có giỏ chứa 3 nhóm. Điều này dẫn đến hiện tượng Data Skew nhẹ (lệch dữ liệu), một số task sẽ phải làm việc nhiều hơn task khác.

---

4. **Tại sao lại có sự khác biệt này?**

Sự khác biệt này thường đến từ việc cấu hình tham số `spark.sql.shuffle.partitions`:

**Ở kế hoạch 1**, có thể bạn (hoặc hệ thống) đã set `set("spark.sql.shuffle.partitions", "4")`.

**Ở kế hoạch 2**, con số này được set là `10`.

**Lời khuyên:** Trong trường hợp cụ thể này (có 10 groups), Kế hoạch 2 tốt hơn vì nó tận dụng tối đa sự phân bổ đều của dữ liệu và tăng khả năng xử lý song song.

## 5️⃣ Pandas?

In [18]:
pdf.groupby("group_id").sum()
# print(pdf.columns)

,name,balance
id,,
1,Alice,1000
2,Bob,2000
3,Charlie,3000


> ❌ RAM chết nếu lớn



Để so sánh một cách công bằng giữa Spark và Pandas khi xử lý dữ liệu lớn, chúng ta cần lưu ý rằng:

Pandas: Chạy trên RAM và sử dụng đơn nhân (single-core) là chính. Nếu dữ liệu vượt quá RAM, nó sẽ bị lỗi (OOM).

Spark: Phân tán dữ liệu ra nhiều phân vùng (partitions) và chạy song song trên nhiều nhân/nhiều máy.

Dưới đây là mã nguồn hoàn chỉnh để bạn chạy trên môi trường có cả Spark và Pandas (như Google Colab hoặc máy local).

---

### Code So Sánh Hiệu Năng GroupBy

In [19]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand
import time

# 1. Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("SparkVsPandas") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Cấu hình số lượng bản ghi (Ví dụ: 10 triệu dòng)
n_rows = 10_000_000

print(f"--- Đang tạo tập dữ liệu {n_rows} dòng ---")

# 2. Tạo dữ liệu mẫu bằng Spark (để tận dụng tốc độ song song khi tạo data lớn)
spark_df = spark.range(0, n_rows) \
    .withColumn("group_id", (col("id") % 100).cast("string")) \
    .withColumn("amount", rand(seed=42) * 100)

# Chuyển sang Pandas để so sánh (Lưu ý: Máy phải đủ RAM để chứa n_rows dòng này)
pdf = spark_df.toPandas()

print("--- Bắt đầu đo lường ---")

# --- THỰC THI VỚI PANDAS ---
start_pd = time.time()
pd_result = pdf.groupby("group_id")["amount"].sum()
end_pd = time.time()
pandas_time = end_pd - start_pd
print(f"Thời gian Pandas thực thi: {pandas_time:.4f} giây")

# --- THỰC THI VỚI SPARK ---
# Lưu ý: Trong Spark phải dùng một Action (như collect hoặc show) để kích hoạt tính toán
start_spark = time.time()
spark_result = spark_df.groupBy("group_id").sum("amount").collect()
end_spark = time.time()
spark_time = end_spark - start_spark
print(f"Thời gian Spark thực thi: {spark_time:.4f} giây")

# 3. So sánh kết quả
print("\n--- Tóm tắt ---")
if pandas_time < spark_time:
    print(f"Kết quả: Pandas nhanh hơn Spark {spark_time/pandas_time:.2f} lần (do dữ liệu vẫn vừa RAM và không mất chi phí quản lý cluster)")
else:
    print(f"Kết quả: Spark nhanh hơn Pandas {pandas_time/spark_time:.2f} lần")

# Đóng Spark Session
# spark.stop()

--- Đang tạo tập dữ liệu 10000000 dòng ---


Py4JJavaError: An error occurred while calling o129.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.execution.SparkPlan$$anon$1._next(SparkPlan.scala:429)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.getNext(SparkPlan.scala:440)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.getNext(SparkPlan.scala:426)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at scala.collection.IterableOnceOps.foreach(IterableOnce.scala:619)
	at scala.collection.IterableOnceOps.foreach$(IterableOnce.scala:617)
	at org.apache.spark.util.NextIterator.foreach(NextIterator.scala:21)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1(SparkPlan.scala:463)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1$adapted(SparkPlan.scala:462)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$3637/0x00007c8fecea33d0.apply(Unknown Source)
	at scala.collection.ArrayOps$.foreach$extension(ArrayOps.scala:1324)
	

## Phân Tích Sự Khác Biệt
Khi bạn chạy code trên với các kích thước dữ liệu khác nhau, bạn sẽ thấy kết quả rất thú vị:

|**Kích thước dữ liệu**|**Ai sẽ thắng?**|**Tại sao?**|
|----------------------|----------------|------------|
|Nhỏ (< 1 triệu dòng)|Pandas|"Spark mất thời gian ""khởi động"" (overhead) để lập kế hoạch thực thi và chia nhỏ công việc."|
|Lớn (10 - 100 triệu dòng)|Spark|Spark tận dụng đa nhân (multi-core) để xử lý song song các partitions. Pandas bắt đầu bị nghẽn ở 1 nhân CPU.|
|Rất lớn (> RAM máy)|Spark|Pandas sẽ bị lỗi Out of Memory (OOM). Spark có thể ghi dữ liệu tạm xuống đĩa (spill to disk) để tiếp tục chạy.|

---

2. Về Cơ Chế Thực Thi

Pandas (Eager Evaluation): Khi bạn gọi .sum(), nó tính toán ngay lập tức.

Spark (Lazy Evaluation): Khi bạn gọi .groupBy().sum(), Spark chỉ ghi lại "kế hoạch" (như cái Explain bạn xem ở câu trước). Nó chỉ thực sự chạy khi bạn gọi .collect() hoặc .show().

3. Tại sao lỗi KeyError của bạn xảy ra?

Trong code Spark, nếu bạn tạo cột bằng id % 10 AS group_id, khi dùng toPandas(), hãy đảm bảo tên cột không bị viết hoa hoặc thay đổi. Lỗi KeyError bạn gặp lúc trước thường do:

Bạn chưa gọi toPandas() mà đã dùng cú pháp của Pandas trên đối tượng Spark.

Tên cột bị thay đổi (ví dụ: sum(amount) thay vì amount).

Lưu ý quan trọng: Nếu bạn đang chạy trên một máy cá nhân (Local), Spark đôi khi chậm hơn Pandas vì chi phí quản lý dữ liệu phân tán trên cùng một máy là khá lớn. Spark chỉ thực sự "vô đối" khi chạy trên một Cluster nhiều máy.


## 6️⃣ Thực tế

*	80% incident Spark = shuffle
*	Bank rất sợ shuffle uncontrolle

# 🟡 LAB 4 – JOIN & BROADCAST (HAY CHẾT NHẤT)

## 🎯 Mục tiêu
*	Hiểu join strategy
*	Tránh OOM

---

1️⃣ Data

In [20]:
orders = spark.range(0, 1_000_000).select(
    (F.col("id") % 1000).alias("customer_id"),
    F.rand().alias("amount")
)

customers = spark.range(0, 1000).select(
    F.col("id").alias("customer_id"),
    F.lit("VN").alias("country")
)

## 2️⃣ Join thường

In [21]:
orders.join(customers, "customer_id").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#35L, amount#36, VN AS country#39]
   +- BroadcastHashJoin [customer_id#35L], [customer_id#38L], Inner, BuildRight, false
      :- Filter isnotnull(customer_id#35L)
      :  +- Project [(id#34L % 1000) AS customer_id#35L, rand(6594456756986365534) AS amount#36]
      :     +- Range (0, 1000000, step=1, splits=2)
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=108]
         +- Project [id#37L AS customer_id#38L]
            +- Range (0, 1000, step=1, splits=2)




## 3️⃣ Broadcast

In [22]:
from pyspark.sql.functions import broadcast

orders.join(broadcast(customers), "customer_id").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#35L, amount#36, VN AS country#39]
   +- BroadcastHashJoin [customer_id#35L], [customer_id#38L], Inner, BuildRight, false
      :- Filter isnotnull(customer_id#35L)
      :  +- Project [(id#34L % 1000) AS customer_id#35L, rand(6594456756986365534) AS amount#36]
      :     +- Range (0, 1000000, step=1, splits=2)
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=139]
         +- Project [id#37L AS customer_id#38L]
            +- Range (0, 1000, step=1, splits=2)




## 4️⃣ Giải thích
*	Broadcast = gửi table nhỏ đến mọi executor
*	Nhanh nhưng OOM nếu sai


## Thực tế ví dụ trên:

Điểm thú vị nhất trong hai bản explain này là: Chúng hoàn toàn giống hệt nhau.

Dù ở trường hợp thứ hai bạn có dùng hàm broadcast() một cách tường minh, nhưng Spark đủ thông minh để tự động thực hiện điều đó ở trường hợp thứ nhất. Dưới đây là phân tích chi tiết:

1. **Tại sao hai kế hoạch lại giống hệt nhau?**

Trong Spark, có một cấu hình tên là `spark.sql.autoBroadcastJoinThreshold` (mặc định là 10MB).

**Bảng customers:** Chỉ có 1.000 dòng với dữ liệu rất nhẹ (một cột Long và một cột String "VN"). Kích thước này nhỏ hơn rất nhiều so với ngưỡng 10MB.

**Cơ chế tự động:** Khi bạn thực hiện Join, bộ tối ưu hóa **Catalyst** của Spark sẽ kiểm tra kích thước các bảng. Vì bảng customers rất nhỏ, Spark tự động chuyển nó thành **BroadcastHashJoin** để tối ưu hiệu suất mà không cần bạn phải ra lệnh bằng hàm broadcast().

---

2. **Phân tích các thành phần chung trong Physical Plan**

Cả hai bản kế hoạch đều có các bước then chốt sau:

**BroadcastHashJoin:** Đây là kiểu Join nhanh nhất trong Spark. Thay vì phải xáo trộn (Shuffle) cả hai bảng lớn qua mạng, Spark lấy bảng nhỏ (customers), gửi bản sao của nó đến tất cả các Executor.

**BuildRight:** Spark chọn bảng bên phải (là customers) để làm bảng bị "broadcast" (bảng dùng để xây dựng Hash Table).

**BroadcastExchange:** Đây là bước vật lý thực hiện việc gửi dữ liệu bảng nhỏ đi khắp các node.

**Filter isnotnull:** Spark tự động thêm bước kiểm tra null trên khóa Join để loại bỏ các dữ liệu thừa ngay từ đầu, giúp tăng tốc quá trình.

---

3. **Quy trình hoạt động của Broadcast Hash Join**

Driver thu thập dữ liệu từ bảng nhỏ (customers).

**Driver** gửi bản sao bảng này đến mọi Executor đang giữ các phân vùng (partitions) của bảng lớn (orders).

Mỗi Executor thực hiện Join ngay tại chỗ (local join) mà không cần di chuyển dữ liệu của bảng orders.

---

4. **Khi nào thì 2 câu lệnh này sẽ cho kết quả khác nhau?**

Bạn sẽ thấy sự khác biệt nếu bảng customers lớn hơn ngưỡng mặc định (ví dụ 100MB):

**Nếu không dùng broadcast():** Spark sẽ thực hiện SortMergeJoin. Đây là kiểu Join yêu cầu Shuffle cả hai bảng. Bạn sẽ thấy bước Exchange hashpartitioning xuất hiện cho cả hai bên trong bản explain. Việc này tốn tài nguyên mạng và chậm hơn.

**Nếu có dùng broadcast():** Bạn "ép" Spark phải dùng Broadcast Join bất kể bảng đó lớn hơn 10MB (miễn là nó vẫn vừa bộ nhớ của Driver và Executor).

---

**Tóm tắt so sánh:**

|**Đặc điểm**|**Cách 1 (Tự động)**|**Cách 2 (Dùng broadcast())**|
|------------|--------------------|-----------------------------|
|Physical Plan|BroadcastHashJoin|BroadcastHashJoin|
|Hiệu suất|Tối ưu nhất|Tối ưu nhất|
|Tính an toàn|An toàn (Chỉ broadcast nếu bảng nhỏ)|"Có rủi ro (Nếu bảng customers đột ngột lớn lên, có thể gây lỗi hết bộ nhớ - OOM)"|

**Lời khuyên:** Bạn nên tin tưởng vào bộ tối ưu hóa của Spark ở trường hợp dữ liệu nhỏ. Chỉ nên dùng hàm broadcast() khi bạn biết chắc chắn bảng đó nhỏ nhưng Spark không tự nhận diện được (ví dụ dữ liệu lấy từ một query phức tạp mà Spark không tính toán trước được kích thước).

Để thấy sự khác biệt rõ rệt nhất mà không cần phải tạo ra hàng GB dữ liệu (làm treo máy), cách tốt nhất là chúng ta sẽ điều chỉnh tham số cấu hình `spark.sql.autoBroadcastJoinThreshold`.

Khi cấu hình này được đặt về `-1`, Spark sẽ bị cấm sử dụng **Broadcast Join** và buộc phải dùng **SortMergeJoin**. Đây chính là cách các hệ thống Big Data xử lý khi cả hai bảng đều cực kỳ lớn.

---

## 1. Code thực thi chi tiết

In [23]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Khởi tạo Spark
spark = SparkSession.builder.appName("SortMergeJoinDemo").getOrCreate()

# 1. Tạo dữ liệu mẫu
orders = spark.range(0, 1000000).select(
    (F.col("id") % 1000).alias("customer_id"),
    F.rand().alias("amount")
)

customers = spark.range(0, 1000).select(
    F.col("id").alias("customer_id"),
    F.lit("VN").alias("country")
)

print("=== 1. TRƯỜNG HỢP MẶC ĐỊNH (BROADCAST JOIN) ===")
# Spark tự nhận diện bảng customers nhỏ nên dùng Broadcast
orders.join(customers, "customer_id").explain()

print("\n" + "="*50 + "\n")

print("=== 2. TRƯỜNG HỢP ÉP DÙNG SORT MERGE JOIN ===")
# Tắt tính năng tự động Broadcast bằng cách set threshold về -1
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

orders.join(customers, "customer_id").explain()

# Reset lại cấu hình về mặc định (10MB) sau khi test xong
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10 * 1024 * 1024)

=== 1. TRƯỜNG HỢP MẶC ĐỊNH (BROADCAST JOIN) ===
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#41L, amount#42, VN AS country#45]
   +- BroadcastHashJoin [customer_id#41L], [customer_id#44L], Inner, BuildRight, false
      :- Filter isnotnull(customer_id#41L)
      :  +- Project [(id#40L % 1000) AS customer_id#41L, rand(-396041844424334531) AS amount#42]
      :     +- Range (0, 1000000, step=1, splits=2)
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=170]
         +- Project [id#43L AS customer_id#44L]
            +- Range (0, 1000, step=1, splits=2)




=== 2. TRƯỜNG HỢP ÉP DÙNG SORT MERGE JOIN ===
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [customer_id#41L, amount#42, VN AS country#45]
   +- SortMergeJoin [customer_id#41L], [customer_id#44L], Inner
      :- Sort [customer_id#41L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(customer_id#41L, 10), ENSURE_RE

## 2. So sánh 2 bản Explain mới

Khi bạn chạy code trên, ở Trường hợp 2, bạn sẽ thấy một kế hoạch thực thi hoàn toàn khác và phức tạp hơn nhiều:

### Các điểm mới xuất hiện trong SortMergeJoin:

**Exchange hashpartitioning:** Spark thực hiện Shuffle. Nó băm (hash) customer_id của cả hai bảng và gửi các dòng có cùng mã băm về cùng một node. Đây là bước tốn kém nhất vì dữ liệu phải chạy qua mạng.

`Sort [customer_id#... ASC NULLS FIRST]`: Sau khi dữ liệu về cùng một node, Spark sắp xếp chúng theo thứ tự tăng dần của customer_id.

**SortMergeJoin:** Cuối cùng, Spark chỉ cần quét qua hai danh sách đã sắp xếp để khớp các cặp dữ liệu (giống như việc bạn so sánh hai danh sách danh bạ đã xếp theo bảng chữ cái).

---

### 3. Bảng so sánh nhanh

|**Đặc điểm**|**Broadcast Hash Join (Trường hợp 1)**|**Sort Merge Join (Trường hợp 2)**|
|------------|---------------------------------------|----------------|
|Shuffle|Không có. Tiết kiệm băng thông mạng.|Có. Phải di chuyển dữ liệu của cả 2 bảng.|
|Sắp xếp (Sort)|Không cần.|Bắt buộc. Tốn thêm CPU và Disk.|
|Giới hạn dữ liệu|Một bảng phải đủ nhỏ để nhét vừa RAM.|Không giới hạn. Có thể join 2 bảng hàng Terabyte.|
|Tốc độ|Rất nhanh.|Chậm hơn (do tốn công Shuffle & Sort).|

---

#### Tại sao chúng ta cần biết điều này?

Trong thực tế, nếu bạn thấy Spark chạy một câu lệnh Join quá chậm (vài tiếng đồng hồ), hãy kiểm tra `.explain()`.

Nếu nó đang dùng *SortMergeJoin* cho một bảng mà bạn biết là nó nhỏ, hãy dùng *broadcast()* để ép nó chạy nhanh hơn.

Nếu nó đang dùng ***BroadcastJoin** mà bị lỗi **Out of Memory (OOM)**, hãy tăng bộ nhớ cho Driver hoặc ép nó quay về **SortMergeJoin**.

## 5️⃣ Pandas?

❌ Join lớn = chết

---

## 6️⃣ Thực tế
*	Broadcast sai = incident
*	Bank thường disable auto broadcast

# 🔴 LAB 5 – SKEW & SALTING (BANK GRADE)

## 🎯 Mục tiêu
*	Hiểu skew
*	Fix skew đúng cách

---

## 1️⃣ Data skew


In [24]:
df = spark.range(0, 1_000_000).select(
    F.when(F.rand() < 0.7, 1).otherwise(F.col("id") % 1000).alias("customer_id"),
    F.rand().alias("amount")
)

## 2️⃣ GroupBy → chậm

In [25]:
df.groupBy("customer_id").sum("amount").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[customer_id#47L], functions=[sum(amount#48)])
   +- Exchange hashpartitioning(customer_id#47L, 10), ENSURE_REQUIREMENTS, [plan_id=223]
      +- HashAggregate(keys=[customer_id#47L], functions=[partial_sum(amount#48)])
         +- Project [CASE WHEN (rand(-5359486187627201171) < 0.7) THEN 1 ELSE (id#46L % 1000) END AS customer_id#47L, rand(-1491132137368338991) AS amount#48]
            +- Range (0, 1000000, step=1, splits=2)




## 3️⃣ Salting

In [26]:
df2 = df.withColumn(
    "salt",
    (F.rand() * 10).cast("int")
)

df2.groupBy("customer_id", "salt").sum("amount") \
   .groupBy("customer_id").sum("sum(amount)")

DataFrame[customer_id: bigint, sum(sum(amount)): double]

## 4️⃣ Giải thích
*	Chia hot key
*	Phân tán task

---

## 5️⃣ Thực tế
*	Skew = design issue
*	Retry không cứu được



## Phân tích:

Ví dụ trên đưa ra là một kịch bản kinh điển về Data Skew (Lệch dữ liệu) trong Big Data. Khi `70%` dữ liệu tập trung vào một giá trị (customer_id = 1), nó sẽ tạo ra hiện tượng **"nút thắt cổ chai"**.

---

## 1. Phân tích Physical Plan ban đầu (Bị Skew)

Trong bản explain của bạn, Spark thực hiện theo quy trình:

**PartialAggregate:** Gom nhóm tạm thời trên từng partition.

**Exchange hashpartitioning:** Đây là nơi thảm họa xảy ra. Spark dùng hàm Hash để quyết định dữ liệu đi về partition nào: `hash(customer_id) % 10`.

Vì `70%` dòng có `customer_id = 1`, nên toàn bộ `700.000` dòng này sẽ bị đẩy vào cùng 1 partition vật lý.

**HashAggregate (Final):** Trong khi **9 Executor** khác có thể chỉ xử lý vài chục nghìn dòng và xong trong 1 giây, thì 1 Executor "số nhọ" phải xử lý 700.000 dòng.

**Hệ quả:** Tổng thời gian chạy của cả Job bằng thời gian chạy của Task chậm nhất đó. Thậm chí có thể gây lỗi **Out Of Memory (OOM)** tại Executor đó.

## 2. Phân tích giải pháp Salting (Muối dữ liệu)

Giải pháp bạn đưa ra là chia nhỏ cái "nút thắt" đó ra.

### Cơ chế:

#### Bước 1:
Thêm một số ngẫu nhiên (salt từ 0-9) vào khóa join/groupby. Khóa mới bây giờ là (1, 0), (1, 1), ..., (1, 9).

#### Bước 2:
`GroupBy` theo cụm (customer_id, salt). Thay vì 1 task xử lý 700k dòng, bây giờ 10 task xử lý, mỗi task ~70k dòng. Dữ liệu được trải đều ra các core.

#### Bước 3:
GroupBy lần nữa theo customer_id để cộng dồn kết quả cuối cùng từ 10 cụm nhỏ kia.

## 3. So sánh và Hướng xử lý tốt nhất

Mặc dù Salting giải quyết được Skew, nhưng nó làm code phức tạp hơn và tạo ra 2 lần Shuffle. Trong Spark hiện đại (3.0 trở đi), chúng ta có những lựa chọn sau:

### Cách 1: Tận dụng AQE (Adaptive Query Execution) - Ưu tiên số 1

Nếu bạn dùng Spark 3.x, hãy bật tính năng tự động xử lý Skew. Spark sẽ tự phát hiện partition nào quá lớn và chia nhỏ nó ra cho bạn mà không cần sửa code.

```python
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true") # Chủ yếu cho Join
```

**Lưu ý:** AQE hỗ trợ cực tốt cho Join, nhưng với GroupBy, cơ chế Partial Aggregate của Spark thường đã xử lý khá tốt trừ khi số lượng key quá ít.

### Cách 2: Salting (Nếu AQE không đủ cân) - Ưu tiên số 2

Bạn chỉ nên dùng ***Salting*** khi:

Dữ liệu cực lớn và cực lệch.

Bạn thấy một vài Task chạy mãi không xong trong khi các Task khác đã hoàn thành từ lâu.

**Code tối ưu cho Salting:** Thay vì dùng rand(), bạn có thể dùng (id % 10) để ổn định hơn (deterministic).

# 🔥 LAB 6 – WRITE GOLD LAYER (BANK STYLE)

## 🎯 Mục tiêu
*	Idempotent
*	Predictable

---

## 1️⃣ Write đúng



In [ ]:
(df
 .repartition(4)
 .write
 .mode("overwrite")
 .partitionBy("date")
 .parquet("/gold/orders"))

## 2️⃣ Giải thích
	•	Không append lung tung
	•	Có rollback
	•	Audit được

--

## 3️⃣ Pandas?

❌ Không đủ an toàn

---

## 🧰 CÔNG CỤ THỰC TẾ (BANK)
	•	Spark UI
	•	Dataproc / EMR
	•	Airflow / Composer
	•	Delta / Iceberg
	•	Great Expectations (data quality)
	•	OpenLineage / Atlas
  

# 🟢 LAB 1 – DATAFRAME FOUNDATION (ĐI CHẬM – HIỂU SÂU)

## Mục tiêu LAB 1

*	Hiểu Spark DataFrame là gì (so với Pandas)
*	Hiểu lazy evaluation
*	Hiểu transform vs action
*	Không code máy móc

---

## 0️⃣ Chuẩn bị (luôn có trong mọi notebook)

```python
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
```

* `functions as F`
> → convention industry, 99% code Spark ngoài đời dùng F

---

## 1️⃣ Tạo SparkSession (trái tim của Spark)

```python
spark = SparkSession.builder \
    .appName("lab1_dataframe_basic") \
    .getOrCreate()
```

### 🧠 Giải thích
*	SparkSession = entry point
*	Mọi DataFrame đều sống trong SparkSession
*	Không có SparkSession → không có Spark

> 📌 Trong Pandas không có khái niệm này

---

## 2️⃣ Tạo DataFrame thủ công (cách dễ hiểu nhất)

```python
data = [
    (1, "Alice", 1000),
    (2, "Bob", 2000),
    (3, "Charlie", 3000)
]

columns = ["id", "name", "balance"]

df = spark.createDataFrame(data, columns)
```

### 🧠 So sánh Pandas

```python
import pandas as pd
pdf = pd.DataFrame(data, columns=columns)
```

|**Pandas**|**Spark**|
|----------|---------|
|DataFrame| nằm trong RAM|	DataFrame là metadata|
|Nhỏ|	Rất lớn|
|Chạy ngay|	Chưa chạy|


---

## 3️⃣ Nhìn schema (cực kỳ quan trọng)

```python
df.printSchema()
```

### 📌 Output:

```text
root
 |-- id: long
 |-- name: string
 |-- balance: long
```

### 🧠 Vì sao phải xem schema?
*	Spark rất strict về type
*	80% bug Spark = type mismatch
*	Khác Pandas (rất “thoáng”)

---

## 4️⃣ Transformation đầu tiên – select

```python
df_select = df.select("id", "balance")
```

### ⛔ CHƯA CÓ GÌ CHẠY

#### 🧠 Vì sao?
*	Đây là TRANSFORMATION
*	Spark chỉ ghi nhớ:
>- “À, user muốn select 2 cột”

---

## 5️⃣ Transformation thứ hai – filter

```python
df_filtered = df.filter(F.col("balance") > 1500)
```

### ⛔ VẪN CHƯA CHẠY

#### 🧠 Đây là chỗ nhiều người nhầm
*	Spark không giống Pandas
*	Không có CPU, không có RAM dùng ở đây

---

## 6️⃣ ACTION – thời điểm Spark THỰC SỰ chạy

```python
df_filtered.show()
```

### 💥 LÚC NÀY MỚI CHẠY

#### 🧠 Những gì Spark làm lúc này (rất quan trọng)
1.	Build Logical Plan
2.	Optimize (Catalyst)
3.	Build Physical Plan
4.	Chia Stage
5.	Tạo Task
6.	Gửi task tới executor
7.	Chạy

> 👉 Trước show() → không có job

---

## 7️⃣ Action khác – count()

```python
df_filtered.count()
```

### 🧠 Khác show() thế nào?

|**show()**|**count()**|
|----------|-----------|
|Hiển thị|Trả về số|
|Có limit|Full scan|
|Debug|Logic|


> 📌 **count()** rất nguy hiểm với data lớn

---

## 8️⃣ Kiểm tra kế hoạch thực thi (bước architect)

```python
df_filtered.explain()
```

### 👉 Bạn sẽ thấy:
* Logical Plan
*	Physical Plan
*	Không thấy Exchange (vì chưa shuffle)

> 🧠 Architect đọc explain trước khi approve job

---

## 9️⃣ Tổng kết LAB 1 (CỰC QUAN TRỌNG)

### ✅ Bạn cần nhớ chắc:
*	Spark LAZY
*	Transformation ≠ Action
*	DataFrame ≠ dữ liệu thật
*	Spark DataFrame = kế hoạch xử lý dữ liệu

---

## 🧠 CÂU HỎI BẮT BUỘC (bạn tự trả lời)
1.	Vì sao filter() không chạy ngay?
2.	Nếu viết 10 dòng transformation liên tiếp, Spark chạy mấy lần?
3.	Vì sao count() nguy hiểm?
4.	Spark DataFrame có giữ dữ liệu không?
5.	Vì sao Spark sống mà Pandas chết?


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [5]:
spark = SparkSession.builder \
    .appName("lab1_dataframe_basic") \
    .getOrCreate()

In [6]:
data = [
    (1, "Alice", 1000),
    (2, "Bob", 2000),
    (3, "Charlie", 3000)
]

columns = ["id", "name", "balance"]

df = spark.createDataFrame(data, columns)

In [7]:
import pandas as pd
pdf = pd.DataFrame(data, columns=columns)

In [8]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- balance: long (nullable = true)



In [9]:
df_select = df.select("id", "balance")

In [10]:
df_filtered = df.filter(F.col("balance") > 1500)

In [11]:
df_filtered.show()

+---+-------+-------+
| id|   name|balance|
+---+-------+-------+
|  2|    Bob|   2000|
|  3|Charlie|   3000|
+---+-------+-------+



In [12]:
df_filtered.count()

2

In [13]:
df_filtered.explain()

== Physical Plan ==
*(1) Filter (isnotnull(balance#2L) AND (balance#2L > 1500))
+- *(1) Scan ExistingRDD[id#0L,name#1,balance#2L]


